In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [10]:
import glob
path = "./../dist/char*.txt"
contents=[]
for filename in glob.glob(path):
    with open(filename, 'r') as f:
        for line in f:
            contents.append(line.strip('\n'))
print(len(contents))
i=0
while (i<len(contents)-1):
    contents[i]=float(contents[i])
    i+=1
    if ((i+1)%15==0):
        i=i+1

1875


In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [12]:
from random import randint as r

In [13]:
import time

In [14]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.fc1 = nn.Linear(in_features=14, out_features=14)
        self.fc2 = nn.Linear(in_features=14, out_features=4)
        #self.out = nn.Linear(in_features=14, out_features = 4, bias=True)
        
    def forward(self, t):
        #1st layer
        t = self.fc1(t)
        #2nd layer
        t = self.fc2(t)

        #t = F.softmax(t)
        
        return t

In [15]:
net=Network()
params=list(net.parameters())
criterion = nn.MSELoss()
net.zero_grad()
optimizer = optim.SGD(net.parameters(), lr=0.0000000001)
lossDuplicationCounter = 0
prevloss=0
loss = 4
i=0

In [16]:
start=time.time()
while ((loss > 0.0005) and (lossDuplicationCounter<10)):
    a=r(0, (len(contents)/15-1))
    chars = torch.tensor(contents[(a*15):(a*15+14)])
    optimizer.zero_grad()   # zero the gradient buffers
    out = net(chars)
    if (contents[a*14+15]=='Marcin'):
        target = torch.tensor([1., 0., 0., 0.])
    elif (contents[a*14+15]=='Bartek'):
        target = torch.tensor([0., 1., 0., 0.])
    elif (contents[a*14+15]=='Maciek'):
        target = torch.tensor([0., 0., 1., 0.,])
    elif (contents[a*14+15]=='Artur'):
        target = torch.tensor([0., 0., 0., 1.,])
    loss = criterion(out, target)
    if prevloss == loss:
        lossDuplicationCounter +=1
        print('loss duplicate #', lossDuplicationCounter)
    else:
        lossDuplicationCounter = 0
    prevloss = loss
    loss.backward(retain_graph=True)
    #loss.backward()
    optimizer.step()   # Does the update
    if i%1000==0:
        print("\n\niteration: ", i, 
              "\nLoss: ", ((str(loss)).strip('tensor()')).strip(', grad_fn=<MseLossBackward>) '), 
              '\nOutput: ', ((str(out)).strip('tensor(')).strip(', grad_fn=<AddBackward0>)'),
              '\nTarget: ', str(target).strip('tensor(').strip(')'))
    i+=1
print("Learning time: \n\t", time.time()-start)
torch.cuda.empty_cache()



iteration:  0 
Loss:  59209124. 
Output:  [-5536.4751, 11037.2539,  9180.5176,  -283.7117],
 
Target:  [0., 0., 0., 1.]


iteration:  1000 
Loss:  25717.0293 
Output:  [-193.3818,   98.4887,  220.0045,   88.3664] 
Target:  [0., 0., 1., 0.]


iteration:  2000 
Loss:  5587.0610 
Output:  [-77.2180, 103.1244,  76.7099,  -8.4749] 
Target:  [0., 1., 0., 0.]


iteration:  3000 
Loss:  2343.2559 
Output:  [-52.6089,  53.7352,  61.5734,   6.9797] 
Target:  [0., 0., 1., 0.]


iteration:  4000 
Loss:  78.9135 
Output:  [-4.1899, 14.2411,  5.2641, -8.7811] 
Target:  [0., 0., 1., 0.]


iteration:  5000 
Loss:  7765.1948 
Output:  [  97.1089,  -95.6271, -110.8627,   -1.8170] 
Target:  [0., 1., 0., 0.]


iteration:  6000 
Loss:  4774.7817 
Output:  [-73.0220,  83.0346,  83.7021,   5.7053] 
Target:  [0., 0., 1., 0.]


iteration:  7000 
Loss:  5015.5229 
Output:  [-75.0586,  82.1732,  87.7708,  11.6403] 
Target:  [0., 1., 0., 0.]


iteration:  8000 
Loss:  3541.8730 
Output:  [ 66.1358, -67.7360, -7



iteration:  72000 
Loss:  679.4931 
Output:  [-1.5666, 11.0258, 50.7867,  3.2453] 
Target:  [1., 0., 0., 0.]


iteration:  73000 
Loss:  7.8396 
Output:  [0.4900, 1.9408, 6.2160, 0.3810] 
Target:  [0., 0., 1., 0.]


iteration:  74000 
Loss:  93.3741 
Output:  [  4.6633, -13.2091,   0.7897,  12.2159] 
Target:  [0., 1., 0., 0.]


iteration:  75000 
Loss:  460.5731 
Output:  [  3.1594, -14.9486, -40.0100,   3.8368] 
Target:  [0., 0., 0., 1.]


iteration:  76000 
Loss:  2147.6399 
Output:  [  0.5127, -17.5450, -90.8136,  -4.9470] 
Target:  [0., 0., 0., 1.]


iteration:  77000 
Loss:  582.6652 
Output:  [ -0.3000,  -6.1427, -47.4311,  -5.5672] 
Target:  [0., 0., 0., 1.]


iteration:  78000 
Loss:  0.1833 
Output:  [ 0.6941,  0.7553, -0.2799, -0.3367] 
Target:  [0., 1., 0., 0.]


iteration:  79000 
Loss:  58.3652 
Output:  [-2.8738,  7.7504, 13.2787, -3.7903] 
Target:  [0., 0., 1., 0.]


iteration:  80000 
Loss:  150.3968 
Output:  [ 1.3445,  0.7709, 23.7537,  5.9572] 
Target:  [0., 1., 0.



iteration:  146000 
Loss:  71.8049 
Output:  [  6.1453, -11.8512,  -0.9830,  10.9238] 
Target:  [1., 0., 0., 0.]


iteration:  147000 
Loss:  278.0699 
Output:  [ 13.5984, -22.7818,   2.6574,  20.6757] 
Target:  [1., 0., 0., 0.]


iteration:  148000 
Loss:  30.0991 
Output:  [-2.8406,  8.3451,  6.2129, -4.4471] 
Target:  [0., 1., 0., 0.]


iteration:  149000 
Loss:  465.1263 
Output:  [ -4.4817,  -8.2599, -40.9806,  -3.1341] 
Target:  [0., 0., 1., 0.]


iteration:  150000 
Loss:  90.2968 
Output:  [  7.1375, -12.6608,   3.8266,  12.1894] 
Target:  [1., 0., 0., 0.]


iteration:  151000 
Loss:  1215.3125 
Output:  [-20.7263,  50.8842,  18.2218, -39.3177] 
Target:  [0., 0., 1., 0.]


iteration:  152000 
Loss:  158.1113 
Output:  [ -5.7624,   0.8988, -22.5364,  -6.6686] 
Target:  [0., 0., 1., 0.]


iteration:  153000 
Loss:  66.7714 
Output:  [-1.6863,  9.1527, 13.6216, -3.4966] 
Target:  [0., 1., 0., 0.]


iteration:  154000 
Loss:  56.8080 
Output:  [-1.1990,  6.6928, 13.2485, -1.3409]



iteration:  219000 
Loss:  13.8254 
Output:  [-1.0100,  5.6517,  5.1997, -2.1683] 
Target:  [0., 0., 1., 0.]


iteration:  220000 
Loss:  6.0385 
Output:  [ 0.4197,  3.4203,  2.6242, -2.2874] 
Target:  [1., 0., 0., 0.]


iteration:  221000 
Loss:  74.5250 
Output:  [  4.1272, -12.0641,  -7.1346,   8.3278] 
Target:  [0., 0., 1., 0.]


iteration:  222000 
Loss:  5.4392 
Output:  [-0.7312,  3.7924,  3.3227, -1.5442] 
Target:  [0., 1., 0., 0.]


iteration:  223000 
Loss:  50.2169 
Output:  [-5.7546,  6.8236, -8.2929, -8.0663] 
Target:  [0., 1., 0., 0.]


iteration:  224000 
Loss:  3.3829 
Output:  [-0.6203,  2.3695,  3.3558, -0.1002] 
Target:  [0., 1., 0., 0.]


iteration:  225000 
Loss:  65.2586 
Output:  [ -5.4341,   2.0375, -12.9876,  -5.6304] 
Target:  [0., 0., 1., 0.]


iteration:  226000 
Loss:  1.8806 
Output:  [-1.0215,  0.3002, -1.3663, -0.8885] 
Target:  [0., 0., 1., 0.]


iteration:  227000 
Loss:  14.9513 
Output:  [1.0374, 3.0726, 8.0188, 0.1542] 
Target:  [0., 0., 1., 0.]





iteration:  293000 
Loss:  20.2352 
Output:  [ 1.8269, -5.5587, -3.5609,  4.6805] 
Target:  [0., 1., 0., 0.]


iteration:  294000 
Loss:  29.4091 
Output:  [ 4.1018, -7.6070, -0.8106,  7.5029] 
Target:  [0., 0., 0., 1.]


iteration:  295000 
Loss:  24.9784 
Output:  [-5.2317,  5.3617, -4.4342, -5.8187] 
Target:  [0., 1., 0., 0.]


iteration:  296000 
Loss:  40.6126 
Output:  [ 2.5916, -8.3365, -4.1844,  7.1452] 
Target:  [0., 1., 0., 0.]


iteration:  297000 
Loss:  81.0675 
Output:  [ -8.1098,   9.1132,  -6.7143, -10.6371] 
Target:  [1., 0., 0., 0.]


iteration:  298000 
Loss:  30.0136 
Output:  [ 4.1531, -6.7818,  2.8559,  7.3054] 
Target:  [0., 0., 1., 0.]


iteration:  299000 
Loss:  87.4748 
Output:  [ -8.4265,   9.7382,  -6.8432, -10.7146] 
Target:  [0., 0., 0., 1.]


iteration:  300000 
Loss:  101.8718 
Output:  [ 4.4794,  5.9685, 18.9919,  1.4301] 
Target:  [0., 1., 0., 0.]


iteration:  301000 
Loss:  582.5154 
Output:  [ 17.1619, -33.7955, -16.1087,  26.1774] 
Target:  [0.,

...



iteration:  1036000 
Loss:  70.9599 
Output:  [  3.2581, -12.1871,  -4.8221,   9.5290] 
Target:  [0., 0., 1., 0.]


iteration:  1037000 
Loss:  0.5842 
Output:  [ 0.4431,  0.8910,  0.7972, -0.7728] 
Target:  [1., 0., 0., 0.]


iteration:  1038000 
Loss:  1.9818 
Output:  [ 0.4583, -0.9547, -1.5789,  1.1846] 
Target:  [0., 1., 0., 0.]


iteration:  1039000 
Loss:  72.9913 
Output:  [  4.9812, -11.7516,  -8.7850,   8.2026] 
Target:  [0., 0., 0., 1.]


iteration:  1040000 
Loss:  1.6765 
Output:  [ 0.1829,  1.9257,  1.0459, -1.1118] 
Target:  [1., 0., 0., 0.]


iteration:  1041000 
Loss:  28.5339 
Output:  [-2.7265,  8.0089,  4.9526, -5.1899] 
Target:  [0., 0., 1., 0.]


iteration:  1042000 
Loss:  10.2905 
Output:  [ 1.9946, -4.6144, -2.8066,  3.8309] 
Target:  [0., 0., 0., 1.]


iteration:  1043000 
Loss:  141.4559 
Output:  [  5.8119, -15.9776, -10.7193,  11.3535] 
Target:  [0., 1., 0., 0.]


iteration:  1044000 
Loss:  0.8207 
Output:  [ 0.0799, -0.2740, -0.3065,  1.2487] 
Target:  



iteration:  1110000 
Loss:  2.4025 
Output:  [-0.8125,  3.0521,  1.5365, -1.5420] 
Target:  [0., 1., 0., 0.]


iteration:  1111000 
Loss:  61.2984 
Output:  [ 3.8660, -9.8219, -7.7996,  7.2319] 
Target:  [0., 1., 0., 0.]


iteration:  1112000 
Loss:  0.1041 
Output:  [ 0.0908, -0.4098,  0.5622,  0.2202] 
Target:  [0., 0., 1., 0.]


iteration:  1113000 
Loss:  2.2516 
Output:  [-0.9751,  2.8412,  1.7095, -1.3202] 
Target:  [0., 1., 0., 0.]


iteration:  1114000 
Loss:  5.7169 
Output:  [-0.6675,  3.6493,  1.2799, -2.2653] 
Target:  [1., 0., 0., 0.]


iteration:  1115000 
Loss:  2.5935 
Output:  [-0.8391,  2.2777,  2.4248, -1.5659] 
Target:  [0., 0., 1., 0.]


iteration:  1116000 
Loss:  15.2982 
Output:  [-1.3093,  5.3856,  3.2079, -3.4925] 
Target:  [0., 0., 0., 1.]


iteration:  1117000 
Loss:  9.3766 
Output:  [-1.4571,  4.7350,  2.5270, -3.2606] 
Target:  [0., 0., 1., 0.]


iteration:  1118000 
Loss:  1.8497 
Output:  [-1.0071,  1.8358,  2.2206, -1.2347] 
Target:  [0., 0., 1., 0.]



iteration:  1184000 
Loss:  0.3657 
Output:  [-0.0735,  0.8900,  1.7028, -0.4139] 
Target:  [0., 0., 1., 0.]


iteration:  1185000 
Loss:  1.5948 
Output:  [ 0.9837, -1.0192, -2.0874,  1.1264] 
Target:  [0., 0., 0., 1.]


iteration:  1186000 
Loss:  3.8866 
Output:  [ 0.0562, -2.7096,  0.0799,  2.7120] 
Target:  [0., 0., 1., 0.]


iteration:  1187000 
Loss:  5.3113 
Output:  [-1.0410,  4.1693,  1.7539, -2.6534] 
Target:  [0., 1., 0., 0.]


iteration:  1188000 
Loss:  1.4145 
Output:  [-0.4551,  2.7787,  0.7939, -1.2872] 
Target:  [0., 1., 0., 0.]


iteration:  1189000 
Loss:  8.4894 
Output:  [-1.7939,  3.6005,  2.9722, -1.9903] 
Target:  [0., 0., 0., 1.]


iteration:  1190000 
Loss:  2.7853 
Output:  [-0.2793,  1.6327,  1.7878, -1.2807] 
Target:  [0., 0., 0., 1.]


iteration:  1191000 
Loss:  7.0288 
Output:  [ 1.6580, -2.6891, -2.1746,  2.6511] 
Target:  [0., 1., 0., 0.]


iteration:  1192000 
Loss:  124.1590 
Output:  [  5.3683, -15.8983, -10.2858,  11.4529] 
Target:  [0., 0., 0.,



iteration:  1258000 
Loss:  343.0518 
Output:  [ 12.3958, -24.7418, -15.6790,  17.6091] 
Target:  [0., 1., 0., 0.]


iteration:  1259000 
Loss:  26.1751 
Output:  [ 2.0786, -6.5990, -3.4906,  5.5183] 
Target:  [0., 1., 0., 0.]


iteration:  1260000 
Loss:  6.9704 
Output:  [-1.3203,  3.5131,  3.9759, -2.2228] 
Target:  [0., 0., 1., 0.]


iteration:  1261000 
Loss:  2.1277 
Output:  [ 0.7019, -1.1634, -1.3057,  1.2778] 
Target:  [0., 1., 0., 0.]


iteration:  1262000 
Loss:  0.6910 
Output:  [ 1.3269, -0.5777, -0.7624,  0.7033] 
Target:  [0., 0., 0., 1.]


iteration:  1263000 
Loss:  5.4320 
Output:  [-1.3601,  3.2445,  2.0291, -1.2878] 
Target:  [0., 0., 0., 1.]


iteration:  1264000 
Loss:  195.5853 
Output:  [  9.1384, -19.3063, -11.7086,  14.3626] 
Target:  [1., 0., 0., 0.]


iteration:  1265000 
Loss:  5.2219 
Output:  [-1.1159,  3.1259,  2.3993, -1.0284] 
Target:  [0., 0., 0., 1.]


iteration:  1266000 
Loss:  3.4736 
Output:  [-0.2466,  2.3554,  1.9851, -1.6888] 
Target:  [1., 



iteration:  1332000 
Loss:  3.8502 
Output:  [ 1.5437, -2.3629, -2.9408,  0.9348] 
Target:  [1., 0., 0., 0.]


iteration:  1333000 
Loss:  1.0801 
Output:  [ 1.5244, -1.4791, -0.8815,  1.0394] 
Target:  [1., 0., 0., 0.]


iteration:  1334000 
Loss:  7.2952 
Output:  [-0.4350,  3.9057,  2.4194, -2.4523] 
Target:  [1., 0., 0., 0.]


iteration:  1335000 
Loss:  277.3229 
Output:  [ 10.8758, -23.0763, -13.7907,  17.3802] 
Target:  [0., 0., 0., 1.]


iteration:  1336000 
Loss:  0.7079 
Output:  [-0.1433, -0.2213, -0.2818,  1.1136] 
Target:  [0., 1., 0., 0.]


iteration:  1337000 
Loss:  30.5116 
Output:  [ 3.1648, -7.5133, -5.8824,  5.1291] 
Target:  [1., 0., 0., 0.]


iteration:  1338000 
Loss:  1.9625 
Output:  [-0.7038,  1.8119,  2.7958, -0.9202] 
Target:  [0., 0., 1., 0.]


iteration:  1339000 
Loss:  3.3717 
Output:  [-0.8745,  2.8644,  1.9847, -1.8835] 
Target:  [0., 0., 1., 0.]


iteration:  1340000 
Loss:  3.4925 
Output:  [-0.9363, -0.7750,  1.9899,  2.4460] 
Target:  [0., 1., 0.

In [17]:
print("\n\niteration: ", i, 
              "\nLoss: ", ((str(loss)).strip('tensor()')).strip(', grad_fn=<MseLossBackward>) '), 
              '\nOutput: ', ((str(out)).strip('tensor(')).strip(', grad_fn=<AddBackward0>)'),
              '\nTarget: ', str(target).strip('tensor(').strip(')'))



iteration:  1362830 
Loss:  0.0004 
Output:  [ 0.0261, -0.0162, -0.0233,  1.0165] 
Target:  [0., 0., 0., 1.]


In [18]:
print
i=0
for filename in glob.glob(path):
    #a=r(0, (len(contents)/15-1))
    chars = torch.tensor(contents[(i*15):(i*15+14)])
    #print(str(net(chars)).strip(", grad_fn=<AddBackward0>)").strip("tensor("), "\t : ",contents[i*15+14])
    print(((str(net(chars))).strip('tensor(')).strip(', grad_fn=<AddBackward0>)'), " : ", contents[i*15+14])
    i=i+1

[-0.0449,  0.2055,  0.3601,  1.0004]  :  Artur
[ 0.6968, -1.9054, -1.4262,  1.9736]  :  Bartek
[ 1.7478, -4.6002, -3.6826,  3.9355]  :  Artur
[-0.0766,  0.1849, -0.0410,  0.7674]  :  Artur
[-1.1750,  3.7865,  1.9584, -2.0857]  :  Bartek
[ 1.9729, -5.6835, -3.8490,  4.4744]  :  Bartek
[ 1.6155, -5.3668, -3.1165,  4.9103]  :  Marcin
[-0.7461,  2.2177,  1.9098, -0.2361]  :  Artur
[ 3.1301, -9.6217, -5.1737,  8.1957]  :  Marcin
[-0.6689,  1.9354,  1.1214, -0.3265]  :  Maciek
[ 2.7395, -8.2281, -6.4127,  6.2863]  :  Marcin
[-0.4306,  1.2904,  0.7903,  0.0788]  :  Artur
[-1.9757,  6.1317,  3.4399, -3.6971]  :  Bartek
[ 0.4746, -0.9791, -1.2017,  1.0765]  :  Bartek
[ 0.2451, -1.1822,  0.4380,  2.2986]  :  Maciek
[-0.6850,  2.0936,  1.2389, -0.6678]  :  Bartek
[ 0.2002, -0.7394, -0.0597,  1.6141]  :  Maciek
[-1.4687,  4.2304,  2.7082, -1.9011]  :  Maciek
[  3.3577, -10.3033,  -6.5904,   8.1652]  :  Marcin
[ 1.4737, -4.4081, -2.5127,  3.9284]  :  Bartek
[-2.3209,  7.0329,  4.1235, -4.2020]  :  

In [19]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in net.state_dict():
    print(param_tensor, "\t", net.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
fc1.weight 	 torch.Size([14, 14])
fc1.bias 	 torch.Size([14])
fc2.weight 	 torch.Size([4, 14])
fc2.bias 	 torch.Size([4])
Optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 1e-10, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [139928161172792, 139928161172720, 139928161172648, 139928161172576]}]


In [20]:
torch.save(net, "net.pt")


/home/malbik/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Network. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [22]:
haba = torch.load("net.pt")
haba.eval()


Network(
  (fc1): Linear(in_features=14, out_features=14, bias=True)
  (fc2): Linear(in_features=14, out_features=4, bias=True)
)

In [ ]:
def maxIndex():
    
i=0
ilezg=0
for filename in glob.glob(path):
    tt=torch.tensor(contents[(a*15):(a*15+14)])
    who = contents[i*15+14]
    predykcja = haba(tt)
    predlist = predykcja.toList().index(max(predykcja.toList()))
    predykcjaOsoba=''
    if predlist == 0:
        predykcjaOsoba='Marcin'
    elif predlist == 1:
        predykcjaOsoba='Bartek'
    elif predlist == 2:
        predykcjaOsoba='Maciek'
    elif predlist == 3:
        predykcjaOsoba='Artur'
    else
        print('ERROR!!!')
    print(i, who, ' : \n', str(tt).strip('tensor(').strip(')'), '\n', str(predykcja).strip('tensor(').strip(', grad_fn=<AddBackward0>)'))
    i+=1
    if who == predykcjaOsoba
